# Requesting ERA 5 data

# Step 0
## Istalling the codes

The precipitated water vapor is computed as:

$ PWV = \frac{1}{\rho_w g} \int_{P_f}^0 q(P) dP $

with $q(P)$ the specific humidity as function of the altitude pressure (g of water per Kg of air) at a pressure level, $\rho_w = 1000$ kg/m$^3$ water density, and $g$ is the gravity acceleration.
$q(P)$ is linearly interpolated along the flight path in the grid of longitude, latitude, and time.


To retrieve the ERA 5 data, you need to install their API and open an account of CDS.
First of all, create a conda environment for ERA5:

```
conda create -n ERA5
conda activate ERA5
```

Then, install:

```
conda install cartopy
conda install xarray
```

Then, install cdsapi, the ERA5 API (instructions at: https://cds.climate.copernicus.eu/api-how-to)

```
pip install cdsapi
```

Finally, open an account on CDS:

https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome


At this point, the routines to read the ERA5 data contained in the library fifipy can be installed:

```
git clone https://github.com/darioflute/fifipy.git
pip install -e fifipy
```

# Step 1
## Get the flight path from data on the server

The following file can be copied into flightpath.py in the server and then imported.
If the data are local, it can be imported from fifipy:

```
from fifipy import flightpath
```

and then used locally. Otherwise, one has to copy the following file into the server.

In [4]:
# Create flight path
#!/usr/bin/env python
def flightpath(path, flight, outpath='./'):
    from glob import glob as gb
    from astropy.io import fits
    from astropy.time import Time
    from astropy.table import Table
    import numpy as np
    import re
    import os
    
    files = sorted(gb(path+'*lw.fits'))
    bfiles = sorted(gb(path+'*sw.fits'))
    
    # Sort according to time
    timefile = np.array([os.path.basename(file)[6:12] for file in files])
    s = np.argsort(timefile)
    files = np.array(files)
    bfiles = np.array(bfiles)
    files = files[s]
    if len(bfiles) > 0:
        bfiles = bfiles[s]

    lon = []
    lat = []
    time = []
    date = []
    press = []
    altitude = []
    obj = []
    obstype = []
    za = []
    restwav = []
    brestwav = []
    redshift = []
    
    obsdate_old = ''
    for i, file in enumerate(files):
        # Check if file is manual or has XX in the name
        if re.match(r".*_manual_.*", file) or re.match(r".*XX.*", file):
            pass
        else:
            try:
                with fits.open(file) as hdl:
                    header = hdl['PRIMARY'].header
                    lon.append(header['LON_STA'])
                    lat.append(header['LAT_STA'])
                    press.append(header['HIERARCH STATICAIRPRESS'])
                    obsdate=header['DATE-OBS']
                    obj.append(header['OBJECT'])
                    obstype.append(header['OBSTYPE'])
                    altitude.append(header['HIERARCH BAROALTITUDE'])
                    za.append(header['ZA_START'])
                    try:
                        z = header['REDSHIFT']
                    except:
                        z = 0
                    redshift.append(z)
                    restw = header['RESTWAV']
                    if restw < 0:
                        restw = header['WAVECENT']
                    restwav.append(restw)
                    if obsdate[0:4] == '1970':
                        print(file ,' has year 1970')
                        if obsdate_old != '':
                            seconds = int(obsdate_old[-2:])
                            if seconds < 30:
                                seconds += 30
                            else:
                                seconds = 59
                            obsdate = '{0:s}{1:02d}'.format(obsdate_old[:-2], seconds)
                        else:
                            print('impossible to repair !')
                    date.append(obsdate)
                    unixt = Time(obsdate).unix
                    time.append(unixt)
                    obsdate_old = obsdate
                if len(bfiles) > 0:
                    with fits.open(bfiles[i]) as hdl:
                        header = hdl['PRIMARY'].header
                        restw = header['RESTWAV']
                        if restw < 0:
                            restw = header['WAVECENT']
                        brestwav.append(restw)
            except:
                print(file, ' is corrupted')
        
    lon = np.array(lon)
    lat = np.array(lat)
    time = np.array(time)
    press = np.array(press)
    date = np.array(date)
    obj = np.array(obj)
    obstype = np.array(obstype)
    altitude = np.array(altitude)
    za = np.array(za)
    restwav = np.array(restwav)
    brestwav = np.array(brestwav)
    redshift = np.array(redshift)

    
    # Sort with unixt
    s = np.argsort(time)
    lon = lon[s]
    lat = lat[s]
    time = time[s]
    press = press[s]
    date = date[s]
    obj = obj[s]
    obstype = obstype[s]
    altitude = altitude[s]
    za = za[s]
    if len(bfiles) > 0:
        brestwav = brestwav[s]
    redshift = redshift[s]
        
    # Save in a fits file
    outname = outpath+'/F'+str(flight)+'path.fits'
    hdu = fits.PrimaryHDU()
    hdu.header['Flight'] = flight
    hdu.header['Time_sta'] = date[0]
    hdu.header['Time_end'] = date[-1]
    hdu1 = fits.ImageHDU()
    hdu1.data = lon
    hdu1.header['EXTNAME'] = 'Longitude'
    hdu2 = fits.ImageHDU()
    hdu2.data = lat
    hdu2.header['EXTNAME'] = 'Latitude'
    hdu3 = fits.ImageHDU()
    hdu3.data = press
    hdu3.header['EXTNAME'] = 'Pressure'
    hdu4 = fits.ImageHDU()
    hdu4.data = time
    hdu4.header['EXTNAME'] = 'Time'
    col1 = fits.Column(name='Object', format='20A', array=obj)
    hdu5 = fits.BinTableHDU.from_columns([col1], name='Object')
    col1 = fits.Column(name='Obstype', format='10A', array=obj)
    hdu6 = fits.BinTableHDU.from_columns([col1], name='Obstype')
    hdu7 = fits.ImageHDU()
    hdu7.data = altitude
    hdu7.header['EXTNAME'] = 'Altitude'
    hdu8 = fits.ImageHDU()
    hdu8.data = za
    hdu8.header['EXTNAME'] = 'ZenithAngle'
    hdu9 = fits.ImageHDU()
    hdu9.data = redshift
    hdu9.header['EXTNAME'] = 'Redshift'
    hdu10 = fits.ImageHDU()
    hdu10.data = restwav
    hdu10.header['EXTNAME'] = 'RedRestWavelength'
    if len(bfiles) > 0:
        hdu11 = fits.ImageHDU()
        hdu11.data = brestwav
        hdu11.header['EXTNAME'] = 'BlueRestWavelength'
        hdul = fits.HDUList([hdu, hdu1, hdu2, hdu3, hdu4, hdu5, hdu6, hdu7, hdu8, hdu9, hdu10, hdu11])
    else:
        hdul = fits.HDUList([hdu, hdu1, hdu2, hdu3, hdu4, hdu5, hdu6, hdu7, hdu8, hdu9, hdu10])
    hdul.writeto(outname, overwrite=True)
    hdul.close()

In the server, open an interactive Python session and write the following lines to
retrieve the flight path for flights 803 and 804.

```
source ~/.bashrc
ipython
from flightpath import flightpath
# December flights
for f in range(803,805):
    print(f)
    path = '/persistent_front/missions/2021/*'+str(f)+'/raw/r0/data/si/F*'+str(f)+'*/Data/'
    flightpath(path, f, 'outputdir')  
```

The flight path files will be saved in the directory outputdir.
You can, at this point, transfer these files in you laptop.

# Step 2
## Grab the data from ERA 5

Once the flight path files are stored in a given path, the routine pwvFlight will do the queries to the ERA 5 database and retrieve the satellite data

from fifipy.era5 import pwvFlight

flights = [845,846,847,848]
for f in flights:
    print(f)
    pwvFlight(f, path='.')

# Step 3
## Compute the PWV values for the SOFIA flight path

At this point it is possible to create the PWV values for the flight path and append the estimates to the flight path files.

In [ ]:
from fifipy.era5 import addPWV

flights = [845,846,847,848]
for f in flights:
    print(f)
    addPWV(f)

# Step 4
## Rescale and give list of values

The program addObsWVZ from fifipy can be used for this.